In [9]:
using POMDPs
using Distributions

In [10]:
using POMDPToolbox # for DiscreteBelief, HistoryRecorder, etc
typealias Categorical DiscreteBelief # just a vector of probabilities

POMDPToolbox.DiscreteBelief

In [12]:
importall POMDPs

We will simplify the needle environment by using a grid world which represents the tissue in which the needle would travel. The state of the needle includes s:(x, y, theta) which is the x,y position in the grid and the orientation of the needle tip. By using a Grid world we are locally approximating the state of the needle, so we should be careful about this so that we can scale this to a finer approximation.

In [ ]:

type GridWorldState 
    x::Int64     # x position
    y::Int64     # y position
    theta::Int64 # orientation of needle tip
    bumped::Bool # did we bump the wall?
    done::Bool   # did we puncture the target?
    discount::Float64
end